# ASTs And Other Python Language Services

This Notebook will give an overview of just some of the modules available in the [Language Services](https://docs.python.org/3/library/language.html) section of the standard Python library, along with the [`inspect`](https://docs.python.org/3/library/inspect.html) module (which is in a different section). I haven’t found a use for all the facilities there; I will just mention the ones that I have, or that I could at least concoct reasonable-looking examples of their use.

In [ ]:
import inspect
import ast
import dis


Note the warnings about the code-generation features: there are no guarantees that these will remain substantially unchanged from one CPython version to the next. So don’t rely too much on these details in important libraries or scripts.

Example toy function to play with:

In [ ]:
def mymax(a : int, b : int) -> int :
    "sample function to see what compilation and disassembly look like."
    if a > b :
        return a
    else :
        return b
    #end if
#end mymax


The `inspect` module has a `getsource()` function that will try to retrieve the source for some object, like the above function. This usually seems to work for objects in normal code:

In [ ]:
funcsrc = inspect.getsource(mymax)
print(funcsrc)

The built-in [`compile()`](https://docs.python.org/3/library/functions.html#compile) function can not only create code objects, it can also produce just the intermediate AST form. So these two would be equivalent:

In [ ]:
syntax1 = ast.parse(funcsrc, filename = "<sample>", mode = "exec", type_comments = True)
syntax2 = compile(funcsrc, filename = "<sample>", mode = "exec", flags = ast.PyCF_ONLY_AST | ast.PyCF_TYPE_COMMENTS)

In [ ]:
print("ast.parse() output = %s" % ast.dump(syntax1))
print()
print("compile() output = %s" % ast.dump(syntax2))

(I won’t bother delving too deeply into those blobs of nodes.)

The normal function of `compile()` is to produce a code object, and it can do this from source code or an AST:

In [ ]:
func = compile(syntax1, filename = "<ast>", mode = "exec")


Note that a code object is not a function, and cannot be called as one:

    print("max(%d, %d) = %d\n" % (3, 4, func(3, 4))) # won’t work


You can disassemble the byte code, to see what the compiler has produced:

In [ ]:
dis.dis(func)


Notice the disassembly splits naturally into two parts: the part at the bottom is the actual function code, while the part at the top actually _defines_ the function, or alternatively _creates the function object_.

I can execute this code object using `exec()`, to create a function object inside a specified namespace:

In [ ]:
namespace = {}
exec(func, namespace)


Now, `namespace` has a function I can call:

In [ ]:
print("%s(%d, %d) = %d" % (namespace["mymax"].__name__, 3, 4, namespace["mymax"](3, 4)))


However, along the way, the source code got lost:

In [ ]:
try :
    print("source of mymax function = %s" % repr(inspect.getsource(namespace["mymax"])))
except Exception as err :
    print("inspecting mymax function: %s" % repr(err))
#end try


Note the function object includes a pointer to the code object:

In [ ]:
dis.dis(namespace["mymax"].__code__)

Doing a second `exec()` on the same code object produces a new function object, but it points to the same coce object as the first one:

In [ ]:
ns2 = {}
exec(func, ns2)
ns2["mymax"] is namespace["mymax"], ns2["mymax"].__code__ is namespace["mymax"].__code__